In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

![](https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSJuXt8UtubEvRzv0JUk5RdWILc4-hro_hWhA&usqp=CAU)ichi.pro

#JuliaCon 2021, the largest Julia Programming event in history

26 July 2021 by Logan Kilpatrick

The JuliaCon 2021 organizing committee is proud to share that JuliaCon is off to an incredible start this year with workshops commencing on Tuesday July 20th and going until the conference talks begin on Wednesday July 28th (2021).

https://julialang.org/blog/2021/07/juliacon2021/

#Code by MarketNeutral https://www.kaggle.com/marketneutral/julia-live-on-kaggle/output

In [ ]:
from datetime import datetime
import numpy as np
from IPython.display import Image, display

#Install Julia

In [ ]:
#Code by MarketNeutral https://www.kaggle.com/marketneutral/julia-live-on-kaggle/output

%%bash

JULIA_VERSION="1.6.0"
JULIA_PACKAGES="DataFrames Feather ShiftedArrays RollingFunctions CSV Pipe Plots"

#JULIA_PACKAGES="IJulia BenchmarkTools PyCall PyPlot"
JULIA_PACKAGES_IF_GPU="CUDA"
JULIA_NUM_THREADS=4

JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
wget -nv $URL -O /tmp/julia.tar.gz
tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
rm /tmp/julia.tar.gz
for PKG in `echo $JULIA_PACKAGES`; do
    echo "Installing Julia package $PKG..."
    julia -e 'using Pkg; pkg"add '$PKG'"'
done

#In version 1.4.2 Error: Error during initialization of module CHOLMOD

#In version 1.6.2 we don't have CSV

In [ ]:
# we set the environment variable for number of threads

import os
os.environ['JULIA_NUM_THREADS'] = "4"

In [ ]:
%%bash

julia -e 'println("Hello Julia.")'
julia -e 'using Base.Threads; println(nthreads())'

#Running Julia in Notebook cells

You can use a Jupyter cell magic to run Julia code. This creates a single Julia session which persists over cells.

In [ ]:
#Code by MarketNeutral https://www.kaggle.com/marketneutral/julia-live-on-kaggle/output

%%time
!pip install --quiet julia
import julia
from julia.api import Julia
julia.install()
jl = Julia(compiled_modules=False)  # cannot use precompiled packages with pyjulia on linux :-(
%load_ext julia.magic

#Run arbitrary Julia code in a cell.

In [ ]:
%%julia

using Base.Threads
println(nthreads())

function double(x)
    return x .* 2
end

x = [1, 2, 3];

#The Julia session persists, so we can reference objects and functions from before.

In [ ]:
%%julia

println(x)
double(x)

#Julia and Python interaction

Return the vector to Python as a numpy array. Note the single "%".

In [ ]:
y = %julia double(x)
print(type(y))
print(y)

#Go back and forth between Python and Julia.

In [ ]:
#Code by MarketNeutral https://www.kaggle.com/marketneutral/julia-live-on-kaggle/output

# this is a Python numpy array
z = np.array([7, 8, 9])    

# we pass it to a Julia function and get back a Python numpy array
a = %julia double($z)      

print(type(a))
print(a)

#If you were to use Julia for real in a Kaggle kernel

I imagine the safest thing is to pass objects via caching to file. Julia supports csv, hdf5, etc. of course. Importantly, Julia supports feather for interoperability between pandas and Julia.

Here is a simple example. Say we want to do some expensive feature engineering and take advantage of Julia's easy multithreading. Here is what we will do

load the data inside the Julia session

do our multithreded feature engineering

save it as a feather file

come back to Python and load the feather file and we are off to our ML pipeline.


In [ ]:
#Code by MarketNeutral https://www.kaggle.com/marketneutral/julia-live-on-kaggle/output

%%julia

using CSV, DataFrames, Feather, RollingFunctions
import Base.Threads.@threads

df = CSV.read("../input/cusersmarildownloadspensioncsv/pension.csv")

gdf = groupby(df, :propertyType)
groups_ = Vector{Any}(undef, length(gdf))

@threads for (i, grp) in collect(enumerate(gdf))
    groups_[i] = transform(grp, :price => (x->runmean(x, 20)) => :roll_from_Julia)
end

df_out = vcat(groups_...)

Feather.write("engineered.feather", df_out)

#ValueError: <PyCall.jlwrap (in a Julia function called from Python)

And many things that I simply couldn't get it.

In [ ]:
import pandas as pd

df = pd.read_feather('engineered.feather')
df.head()

#FileNotFoundError: Errno 2 No such file or directory: 'engineered.feather'

#More than one hour changing Julia versions, making searchs trying to fix it. Received only errors and could not read Pension file.

In [ ]:
!apt-get install -y qt5-default

In [ ]:
#Code by MarketNeutral https://www.kaggle.com/marketneutral/julia-live-on-kaggle/output

%%writefile "ilovejulia.jl"

ENV["GKSwstype"] = "nul"

using Plots

@userplot CirclePlot
@recipe function f(cp::CirclePlot)
    x, y, i = cp.args
    n = length(x)
    inds = circshift(1:n, 1 - i)
    linewidth --> range(0, 10, length = n)
    seriesalpha --> range(0, 1, length = n)
    aspect_ratio --> 1
    label --> false
    x[inds], y[inds]
end

n = 400
t = range(0, 2π, length = n)
x = 16sin.(t).^3
y = 13cos.(t) .- 5cos.(2t) .- 2cos.(3t) .- cos.(4t)

anim = @animate for i ∈ 1:n
    circleplot(x, y, i, line_z = 1:n, cbar = false, c = :reds, framestyle = :none)
end when i > 40 && mod1(i, 10) == 5

gif(anim, "ilovejulia.gif", fps = 15)

In [ ]:
#Code by MarketNeutral https://www.kaggle.com/marketneutral/julia-live-on-kaggle/output

%%bash

julia ilovejulia.jl

#After all that, I couldn't read Pension file, nor make the gif. Do I love Julia? 

No need to answer, except if you have any clue to fix it.